In [ ]:
import json
import operator

In [ ]:
pool_depth = 100

In [ ]:
def AvgPrecision(gt, gt_len, sub, index_of_ad, index_of_score):
    score = 100
    seen = []
    num_hits = 0
    num_retrieved = 0
    cumpre = 0
    for entry in sub:
        num_retrieved += 1
        ad_id = entry[index_of_ad]
        # excessive checking of dupes
        if ad_id in seen:
            print "DUPLICATE"
            return None
        seen.append(ad_id)
        # excessive checking of order
        if entry[index_of_score] > score:
            print "ORDER PROBLEM"
            return None
        score = entry[index_of_score]
        if ad_id in gt:
            num_hits += 1
            cumpre += num_hits/float(num_retrieved)
            
    return cumpre/gt_len

In [ ]:
def clean_sub(sub, index_of_ad, index_of_score, expected_length):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    sub.sort(key=operator.itemgetter(index_of_score), reverse=True)
    for entry in sub:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        if entry[index_of_ad] not in seen:
            uniqs.append(entry)
            seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

In [ ]:
gt_file =  '../../../data/post_hoc_annotations/post_hoc_point_fact_combined.json'
    
with open(gt_file, 'r') as f:
    gt = eval(f.read())

In [ ]:
id_pos = 1
score_pos = 2
ans_length = 3 
id_length = 64

clus_type = 'Point Fact'

### To avoid confusing dependecies, we'll get submission data from raw submissions

In [ ]:
submission_path = '../../../data/team_submissions/'

In [ ]:
answers = {}

# NYU

In [ ]:
answers['NYU'] = {}
answers['NYU'][clus_type] = {}

In [ ]:
file_path = submission_path + ('Uncharted/DomainDiscovery/uncharted_NYU_DD.json')
f = open(file_path, 'r')
all_data = eval(f.read())
data = []
for entry in all_data:
    if entry['questionType'] == clus_type:
        data.append(entry)

for qid in gt.keys():
    submissions = []
    for entry in data:
        # We are only evaluating some of them
        if entry['question_id'] == qid:
            submissions = clean_sub(entry['answers'], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['NYU'][clus_type][qid] = submissions

# HG

In [ ]:
answers['HG'] = {}
answers['HG'][clus_type] = {}

In [ ]:
file_path = submission_path + ('Uncharted/DomainDiscovery/uncharted_HG_DD.json')
f = open(file_path, 'r')
all_data = eval(f.read())
data = []
for entry in all_data:
    if entry['questionType'] == clus_type:
        data.append(entry)

for qid in gt.keys():
    submissions = []
    for entry in data:
        # We are only evaluating some of them
        if entry['question_id'] == qid:
            submissions = clean_sub(entry['answers'], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['HG'][clus_type][qid] = submissions

# JPL

In [ ]:
answers['JPL'] = {}
answers['JPL'][clus_type] = {}

In [ ]:
file_path = submission_path + ('Uncharted/DomainDiscovery/uncharted_JPL_DD.json')
f = open(file_path, 'r')
all_data = eval(f.read())
data = []
for entry in all_data:
    if entry['questionType'] == clus_type:
        data.append(entry)

for qid in gt.keys():
    submissions = []
    for entry in data:
        # We are only evaluating some of them
        if entry['question_id'] == qid:
            submissions = clean_sub(entry['answers'], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['JPL'][clus_type][qid] = submissions

# Calculate Results

In [ ]:
clus_type = 'Point Fact'
results = {}
results[clus_type] = {}

for dataset in answers.keys():
    print dataset.lower()
    results[clus_type][dataset] = {}
    for quest in gt.keys():
        results[clus_type][dataset][quest] = {}
        submission = answers[dataset][clus_type][quest]
        results[clus_type][dataset][quest] = {}
        # Setting default to "NA".  Will adjust manually
        results[clus_type][dataset][quest]['pf_metric'] = 'NA'
        # Some datasets have no relevant ads for a question
        if dataset.lower() in gt[quest]['relevant_ads'].keys():
            relevant_ads = gt[quest]['relevant_ads'][dataset.lower()].keys()
            if len(relevant_ads) > 0:
                results[clus_type][dataset][quest]['avg_precision'] = AvgPrecision(relevant_ads, len(relevant_ads),
                                                                                   submission, id_pos, score_pos)
            else:
                results[clus_type][dataset][quest]['avg_precision'] = 'No Relevant Docs Found'
        else:
            results[clus_type][dataset][quest]['avg_precision'] = 'No Relevant Docs Found'

# Manually determine Point Fact metric

In [ ]:
for dataset in results['Point Fact'].keys():
    print ' '
    print dataset
    for qid in results['Point Fact'][dataset].keys():
        # Only need to consider questions with relevant ads
        if results['Point Fact'][dataset][qid]['avg_precision'] != 'No Relevant Docs Found':
            if results['Point Fact'][dataset][qid]['avg_precision'] != 0:
                print ' '
                print qid
                for ad in gt[qid]['relevant_ads'][dataset.lower()].keys():
                    for entry in answers[dataset]['Point Fact'][qid]:
                        if entry[1] == ad:
                            print ' '
                            print ad
                            print 'expected'
                            print gt[qid]['relevant_ads'][dataset.lower()][ad]
                            print 'submitted'
                            print entry[0]

results['Point Fact']['NYU']['1647']['pf_metric'] = 1
results['Point Fact']['NYU']['1707']['pf_metric'] = 1
results['Point Fact']['NYU']['799']['pf_metric'] = 0
results['Point Fact']['NYU']['1339']['pf_metric'] = 0.5

results['Point Fact']['JPL']['217']['pf_metric'] = 0
results['Point Fact']['JPL']['1707']['pf_metric'] = 0.66
results['Point Fact']['JPL']['510']['pf_metric'] = 0.5


results['Point Fact']['HG']['1707']['pf_metric'] = 1
results['Point Fact']['HG']['1038']['pf_metric'] = 0
results['Point Fact']['HG']['799']['pf_metric'] = 0
results['Point Fact']['HG']['510']['pf_metric'] = 0.25
results['Point Fact']['HG']['284']['pf_metric'] = 'NA'


In [ ]:
output_file = 'Uncharted_DD_Point_Fact_Results_REWRITE.json'
with open(output_file, 'w') as outfile:
    json.dump(results, outfile, indent=2)

# DEBUG

In [ ]:
# Spot checking high Average Precision
print "Ground Truth"
for ad in gt['1647']['relevant_ads']['nyu'].keys():
    print ad
print ' '
print "Submission"
for entry in answers['NYU']['Point Fact']['1647']:
    print entry[1]